In [20]:
#!pip install PyPDF2
#nltk.download('punkt')
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\7oda\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [1]:
import PyPDF2
from PyPDF2 import PdfFileReader
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
import numpy as np
import ast
#from ast import literal_eval

In [111]:
def files():
    """
    
        return: list of files in the directory 
    
    """
    directory = "C:\\Users\\7oda\\New folder\\Query-Optimization\\files\\"
    files_name = []
    for sample in os.listdir(directory):
        files_name.append(sample)
    return files_name

In [112]:
def order(files_list):
    """
        files_list: list of files we will work on

        return: ordered files_list
    """
    temp = 0
    str_temp=''
    for i in range(len(files_list)):
        pdf = open("files/"+files_list[i],"rb")
        pdf_reader = PyPDF2.PdfFileReader(pdf)
        if(pdf_reader.numPages > temp):
            temp = pdf_reader.numPages 
            str_temp = files_list[i]
            del files_list[i]
            files_list.insert(0, str_temp)
            #print("found"+ files_list[i])
            
    return files_list

In [132]:
def reading():
    """
    
        return: a dataframe with all the files in columns and the pages of each file in rows
    
    """
        
    print("=============in the reading =============")

    #files_list =[]
    files_list = files()
    #make the bigger document in the begenning of the list
    files_list = order(files_list)
    
    #creating a dataframe to hold the docs
    df= pd.DataFrame()#

    for j in range(len(files_list)):
        page_list=[]
        print(files_list[j])
        pdf = open("files/"+files_list[j],"rb")
        pdf_reader = PyPDF2.PdfFileReader(pdf)

        print("num of pages -> "+ str(pdf_reader.numPages))

        for i in range (pdf_reader.numPages):
            page = pdf_reader.getPage(i)
            pdf_words = page.extractText()
            page_list.append(pdf_words)
            #print(page_list)
        df[files_list[j]] = pd.Series(page_list)
        


    #closing the pdf file
    pdf.close()
    return df

In [114]:
def normalize(df):
    """
        df: a dataframe of all the files

        return: normalized dataframe with pages index
    """
    print("=============in the normalize =============")
    df['pages'] = df.index
    #this cell runs only one time
    # if you want to run again go back from the previous cell

    files_list = files()
    stop = stopwords.words('english')
    for i in  files_list:
        # convert to lower case
        df[i] = df[i].str.lower()
        # removing puncituations
        df[i] = df[i].str.replace('[^\w\s]','')
        #convert to string
        df[i] = df[i].apply(str)
    return df

In [115]:
def expanding(df):
    """
        df: a normalized dataframe of all the files

        return: a dataframe with each row represent a line not page, and make lines index
    """
    print("=============in the expanding =============")
    new_df=pd.DataFrame()
#    global word_list 
#    global index_list 
    #loop on each column or file
    for i in range (len(df.columns)-1):
        word_list = []
        index_list = []
        page_list = []
        index=0
        print("in the file #"+str(i))
        print("===================================")
        #loope on each cell or page
        for j in range(len(df.index)):
            ptr=-1
            #print(j)
            if (df.iloc[j][i] == 'nan'):
                break
            #loop on each element on the cell or character
            for k in range(len(df.iloc[j][i])):
                if (df.iloc[j][i][k] == '\n'):
                    index_list.append(index)
                    word_list.append(df.iloc[j][i][ptr+1:k])
                    page_list.append(df.iloc[j][len(df.columns)-1])
                    ptr = k
                    index=index+1
                    #print(k)
                    
        new_df[str(i)] = pd.Series(word_list)
        new_df['file '+ str(i)+' indexes'] = pd.Series(index_list)
        new_df['file '+ str(i)+' page numbers'] = pd.Series(page_list)
        print("length of word_list "+str(i)+" is  ---> "+str(len(word_list)))
        
    return new_df

In [130]:
def tokenize(new_df):
    """
        new_df: dataframe of the files

        return: tokenized dataframe
    """
    print("=============in the tokenize=============")
    for i in  range(0,10,1):
        new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(str)
        new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(nltk.word_tokenize)
        #new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(lambda x: [item for item in x if item not in stop])
    return new_df

In [117]:
def return_term_list(new_df):
    """
        new_df: a dataframe of all the files

        return: list of lists containing each token with it's doc_id and token_id
    """
    print("=============in the return_term_list =============")
    all_terms_list = [[]]
    
    #loop on each file
    for i in range (0, len(new_df.columns), 3):
        #loope on each cell or page
        counter = 0
        for j in range(len(new_df.index)):
            if (new_df.iloc[j][i] == ['nan'] or new_df.iloc[j][i] == []):
                continue
            for k in range(len(new_df.iloc[j][i])):
                term_list = []
                #term
                term_list.append(new_df.iloc[j][i][k])
                #doc_id
                term_list.append(i/3)
                #token_pos in the document
                term_list.append(new_df.iloc[j][i+1]+k+counter)
                if(k == 1 and len(new_df.iloc[j][i]) == 2):
                    counter +=1
                elif(k == len(new_df.iloc[j][i])-1):
                    counter += k 
                #page_id
                #term_list.append(new_df.iloc[j][i+2])
                
                #append each term to the all_term_list
                all_terms_list.append(term_list)
            
    return all_terms_list      
#        print(i)

In [129]:
def stop_words_removal(all_terms_list):
    """
        all_terms_list: list of lists containing all the terms in all the files

        return: list of lists after removing stop words
    """
    new_terms_list = [[]]
    print("=============in the stop_words_removal=============")
    stop = stopwords.words('english')

    for i in range(len(all_terms_list)):
        if (all_terms_list[i] != []):
            if not all_terms_list[i][0] in stop:
                new_terms_list.append(all_terms_list[i])#.apply(lambda x: [item for item in x if item not in stop])
    return new_terms_list

In [131]:
df = reading()
df = normalize(df)
df = expanding(df)
df = tokenize(df)
all_terms_list = return_term_list(df)
all_terms_list = stop_words_removal(all_terms_list)

=============in the reading =============
Do the Work_ Overcome Resistance and Get Out of Your Own Way ( PDFDrive ).pdf
57
Artificial Intelligince.pdf
19
7348-dialog-based-interactive-image-retrieval.pdf
11
1905.11946v3.pdf
10
8299-unsupervised-scale-consistent-depth-and-ego-motion-learning-from-monocular-video(1).pdf
11
ImportantTexts.pdf
1
RH_StudyGuide_V2.pdf
32
sample.pdf
2
TextExtractionandCharacterRecognitionform.pdf
5
Visualizing_non-speech_sounds_for_the_deaf.pdf
9
=============in the normalize =============
=============in the expanding =============
in the file #0
length of word_list 0 is  ---> 2484
in the file #1
length of word_list 1 is  ---> 300
in the file #2
length of word_list 2 is  ---> 1418
in the file #3
length of word_list 3 is  ---> 2283
in the file #4
length of word_list 4 is  ---> 1581
in the file #5
length of word_list 5 is  ---> 56
in the file #6
length of word_list 6 is  ---> 554
in the file #7


length of word_list 7 is  ---> 0
in the file #8
length of word_list 8 is  ---> 1037
in the file #9
length of word_list 9 is  ---> 1289
=============in the tokenize=============


TypeError: expected string or bytes-like object

In [121]:
all_terms_list

[[],
 ['work', 0.0, 2],
 ['work', 0.0, 5],
 ['overcome', 0.0, 6],
 ['resistance', 0.0, 7],
 ['get', 0.0, 9],
 ['way', 0.0, 14],
 ['eld', 0.0, 15],
 ['author', 0.0, 16],
 ['war', 0.0, 19],
 ['art', 0.0, 21],
 ['also', 0.0, 22],
 ['steven', 0.0, 24],
 ['pres', 0.0, 25],
 ['eld', 0.0, 26],
 ['fiction', 0.0, 27],
 ['e', 0.0, 28],
 ['profession', 0.0, 29],
 ['killing', 0.0, 30],
 ['rommel', 0.0, 31],
 ['e', 0.0, 32],
 ['afghan', 0.0, 33],
 ['campaign', 0.0, 34],
 ['e', 0.0, 35],
 ['virtues', 0.0, 36],
 ['war', 0.0, 38],
 ['last', 0.0, 39],
 ['amazons', 0.0, 42],
 ['tides', 0.0, 43],
 ['war', 0.0, 45],
 ['gates', 0.0, 46],
 ['fire', 0.0, 48],
 ['e', 0.0, 49],
 ['legend', 0.0, 50],
 ['bagger', 0.0, 52],
 ['vance', 0.0, 53],
 ['n', 0.0, 54],
 ['ction', 0.0, 55],
 ['e', 0.0, 56],
 ['lions', 0.0, 57],
 ['gate', 0.0, 58],
 ['e', 0.0, 59],
 ['authentic', 0.0, 60],
 ['swing', 0.0, 61],
 ['turning', 0.0, 62],
 ['pro', 0.0, 63],
 ['e', 0.0, 64],
 ['warrior', 0.0, 65],
 ['ethos', 0.0, 66],
 ['e', 0.0,

In [5]:
#df['pages'] = df.index
#this cell runs only one time
# if you want to run again go back from the previous cell

#files_list = files()
#stop = stopwords.words('english')
#for i in  files_list:
    # convert to lower case
  #  df[i] = df[i].str.lower()
    # removing puncituations
 #   df[i] = df[i].str.replace('[^\w\s]','')
    #convert to string
#    df[i] = df[i].apply(str)
    

    #splitting new lines on '\n'
#    for j in range(len(df.index)):
#        df[i][j] = df[i][j].replace('\r', ' ').split('\n')
    #df[i] = df[i].apply(literal_eval)    
    #df[i] = df[i].apply(str)

    #df.explode(i)

# convert list of pd.Series then stack it
#    df = (df
     #.set_index(['issue_key','date','pkey','case_count'])['component']
#     .apply(pd.Series)
#     .stack()
#     .reset_index()
#     .reset_index(level=0, drop=True))
     #.rename(columns={0:'component'}))

    #convert to string
    #df[i] = df[i].apply(str)
    
    # tokenize
    #df[i] = df[i].apply(nltk.word_tokenize)
    
    # removing stop words
    #df[i].apply(lambda x: [item for item in x if item not in stop])
    
    #df[i]= df[i].str.splitlines()
    #df[i].apply(lambda x: (word for word in x.str.splitlines()))
    